In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
import tensorflow as tf
#from keras.applications.vgg19 import VGG19
from tensorflow.keras.applications import Xception
from keras.layers import Dense, Flatten
from keras.models import Model
from keras.preprocessing.image import ImageDataGenerator
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.callbacks import EarlyStopping 
from glob import glob
import pickle
from keras.models import Sequential, load_model
from keras.layers import Dropout, Dense, GlobalAveragePooling2D

In [8]:
from sklearn.model_selection import StratifiedShuffleSplit
import cv2
import albumentations as albu
from skimage.transform import resize
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from pylab import rcParams
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from keras.callbacks import Callback, EarlyStopping, ReduceLROnPlateau
import tensorflow as tf
import keras
from keras.models import Sequential, load_model
from keras.layers import Dropout, Dense, GlobalAveragePooling2D
from keras.optimizers import Adam

In [9]:
epochs = 2
height = 224
width = 224
channels = 3
input_shape = (height, width, channels)
#input_shape = [600, 600] + [3]
n_classes = 9

efnb0 = Xception(weights='imagenet', include_top=False, input_shape=input_shape, classes=n_classes)
for layer in efnb0.layers:
        layer.trainable = False

model = Sequential()
model.add(efnb0)
model.add(GlobalAveragePooling2D())
model.add(Dropout(0.5))
model.add(Dense(n_classes, activation='softmax'))

input_shape_new = (None, height, width, channels)
model.build(input_shape_new)

model.summary()

83689472/83683744 [==============================] - 0s 0us/step
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
module_wrapper (ModuleWrappe (None, 7, 7, 2048)        20861480  
_________________________________________________________________
global_average_pooling2d (Gl (None, 2048)              0         
_________________________________________________________________
dropout (Dropout)            (None, 2048)              0         
_________________________________________________________________
dense (Dense)                (None, 9)                 18441     
Total params: 20,879,921
Trainable params: 18,441
Non-trainable params: 20,861,480
_________________________________________________________________


In [10]:
optimizer = Adam(lr=0.0001)

#early stopping to monitor the validation loss and avoid overfitting
early_stop = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=10, restore_best_weights=True)

#reducing learning rate on plateau
rlrop = ReduceLROnPlateau(monitor='val_loss', mode='min', patience= 5, factor= 0.5, min_lr= 1e-6, verbose=1)
#model compiling
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:375: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


In [11]:
train_datagen = ImageDataGenerator(zoom_range = 0.2, 
                                   rescale = 1./255,
                                   shear_range = 0.2,
                                   horizontal_flip = True,
                                   validation_split=0.2)

In [12]:
training_set = train_datagen.flow_from_directory("/content/drive/MyDrive/DL-Project/Colab/dataset",
                                                 target_size = (224, 224),
                                                 batch_size = 32,
                                                 class_mode = 'categorical',
                                                 subset='training')

Found 6670 images belonging to 9 classes.


In [13]:
valid_data_generator = train_datagen.flow_from_directory("/content/drive/MyDrive/DL-Project/Colab/dataset",
                                                 target_size = (224, 224),
                                                 batch_size = 32,
                                                 class_mode = 'categorical',
                                                 subset='validation')

Found 1665 images belonging to 9 classes.


In [14]:
epochs = 10

In [15]:
model_history = model.fit_generator(training_set, validation_data = valid_data_generator, callbacks = [early_stop, rlrop],verbose = 1, epochs = epochs)

#saving the trained model weights as data file in .h5 format
model.save_weights("/content/drive/MyDrive/DL-Project/Colab/Saved Models/Xception/cifar_efficientnetb0_weights.h5")

/usr/local/lib/python3.7/dist-packages/keras/engine/training.py:1915: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/10
209/209 [==============================] - 4522s 21s/step - loss: 2.0854 - accuracy: 0.2351 - val_loss: 1.4947 - val_accuracy: 0.6679
Epoch 2/10
209/209 [==============================] - 1888s 9s/step - loss: 1.3641 - accuracy: 0.6449 - val_loss: 1.1158 - val_accuracy: 0.7748
Epoch 3/10
209/209 [==============================] - 1946s 9s/step - loss: 1.0157 - accuracy: 0.7625 - val_loss: 0.9169 - val_accuracy: 0.7940
Epoch 4/10
209/209 [==============================] - 1833s 9s/step - loss: 0.8229 - accuracy: 0.8070 - val_loss: 0.8021 - val_accuracy: 0.8216
Epoch 5/10
209/209 [==============================] - 1803s 9s/step - loss: 0.7098 - accuracy: 0.8287 - val_loss: 0.7124 - val_accuracy: 0.8234
Epoch 6/10
209/209 [==============================] - 1809s 9s/step - loss: 0.6161 - accuracy: 0.8520 - val_loss: 0.6504 - val_accuracy: 0.8318
Epoch 7/10
209/209 [==============================] - 1792s 9s/step - loss: 0.5549 - accuracy: 0.8646 - val_loss: 0.5943 - val_accuracy

In [16]:
r = model_history

In [26]:
#accuracies
plt.plot(r.history['accuracy'], label='train acc')
plt.plot(r.history['val_accuracy'], label='val acc')
plt.legend()
plt.show()
plt.savefig('/content/drive/MyDrive/DL-Project/Colab/Saved Models/Xception/Xception_acc')

# loss
plt.plot(r.history['loss'], label='train loss')
plt.plot(r.history['val_loss'], label='val loss')
plt.legend()
plt.show()
plt.savefig('/content/drive/MyDrive/DL-Project/Colab/Saved Models/Xception/Xception_loss')


# model.save('/content/drive/MyDrive/DL-Project/Colab/Saved Models/Xception/Xception_model.h5')
model.save('/content/drive/MyDrive/DL-Project/Colab/Saved Models/Xception/Xception_Model')      
model.save_weights('/content/drive/MyDrive/DL-Project/Colab/Saved Models/Xception/Xception_weights_model_1.h5')  

# with open('/content/drive/MyDrive/DL-Project/Colab/Saved Models/Xception/Xception_model.json', 'w') as f:
#     f.write(model.to_json())
 
file = open( "/content/drive/MyDrive/DL-Project/Colab/Saved Models/Xception/Xception_model_acc",'wb') 
pickle.dump(list(r.history['accuracy']), file) 
file.close()

file = open( "/content/drive/MyDrive/DL-Project/Colab/Saved Models/Xception/Xception_model_loss",'wb') 
pickle.dump(list(r.history['loss']), file) 
file.close()  

file = open( "/content/drive/MyDrive/DL-Project/Colab/Saved Models/Xception/Xception_model_val_acc",'wb') 
pickle.dump(list(r.history['val_accuracy']), file) 
file.close() 

file = open( "/content/drive/MyDrive/DL-Project/Colab/Saved Models/Xception/Xception_model_val_loss",'wb') 
pickle.dump(list(r.history['val_loss']), file) 
file.close() 